In [57]:
import pandas
import json
from dateutil import parser
import re

In [58]:
with open('../Dataset/queryLogs.json','r') as log_file:
    log_data = json.load(log_file)
print(log_data)

[{'t': {'$date': '2024-04-04T20:37:10.296+05:30'}, 's': 'I', 'c': 'COMMAND', 'id': 51803, 'ctx': 'conn10', 'msg': 'Slow query', 'attr': {'type': 'command', 'ns': 'test_database.test_collection', 'command': {'find': 'test_collection', 'filter': {'user': '{}', 'password': '{}'}, 'lsid': {'id': {'$uuid': 'e5e23d7e-5367-4829-a104-ca369c49d4dc'}}, '$db': 'test_database'}, 'planSummary': 'EOF', 'planningTimeMicros': 83, 'keysExamined': 0, 'docsExamined': 0, 'nBatches': 1, 'cursorExhausted': True, 'numYields': 0, 'nreturned': 0, 'queryFramework': 'classic', 'reslen': 118, 'locks': {'FeatureCompatibilityVersion': {'acquireCount': {'r': 1}}, 'Global': {'acquireCount': {'r': 1}}}, 'storage': {}, 'cpuNanos': 176371, 'remote': '127.0.0.1:47038', 'protocol': 'op_msg', 'durationMillis': 0}}, {'t': {'$date': '2024-04-04T20:37:10.297+05:30'}, 's': 'I', 'c': 'COMMAND', 'id': 51803, 'ctx': 'conn10', 'msg': 'Slow query', 'attr': {'type': 'command', 'ns': 'test_database.test_collection', 'command': {'find

In [59]:
log_data_frame = pandas.DataFrame(log_data)
log_data_frame.head()

,t,s,c,id,ctx,msg,attr
0,{'$date': '2024-04-04T20:37:10.296+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
1,{'$date': '2024-04-04T20:37:10.297+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
2,{'$date': '2024-04-04T20:37:10.297+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
3,{'$date': '2024-04-04T20:37:10.298+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
4,{'$date': '2024-04-04T20:37:10.298+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."


In [60]:
# Loading other Data

# MongoDB Attribute Columns
with open("../Dataset/MongoDB_attributes_list.json") as MongoDB_attr_data:
    mongo_db_attribute_data = json.load(MongoDB_attr_data)

mongo_db_attribute_columns = mongo_db_attribute_data["mongo_db_attr"]
command_columns = mongo_db_attribute_data["command_columns"]
query_operators = mongo_db_attribute_data["query_operators"]

In [62]:
# Cleaning Data

#Convert T (Timestamp) to Timestamps
log_data_frame_cleaned = pandas.DataFrame()
log_data_frame_cleaned['t'] = log_data_frame['t'].apply(lambda x: parser.parse(x['$date']).timestamp())

#Extract Attr Columns in Nested attr Column
for col in mongo_db_attribute_columns:
    log_data_frame_cleaned[col] = log_data_frame['attr'].apply(lambda x: x.get(col, None))

#Extract Command Columns in Nested command Column
for col in command_columns:
    log_data_frame_cleaned[col] = log_data_frame_cleaned['command'].apply(lambda x: x.get(col, None) if isinstance(x, dict) else None)

#Convert Other Object Columns to str for hashing (For later Operations)
log_data_frame_cleaned[log_data_frame_cleaned.select_dtypes(include=['object']).columns] = log_data_frame_cleaned.select_dtypes(include=['object']).astype(str)

# Find which Particular Query Operator Exists in filter column
for operatorType in query_operators['all_operators']:
    log_data_frame_cleaned[operatorType] = log_data_frame_cleaned['filter'].apply(lambda x:1 if operatorType in x else 0)

# Find which Classes of Query Operators Exists in filter Column
for operatorClass in query_operators:
    log_data_frame_cleaned[operatorClass] = log_data_frame_cleaned['filter'].apply(lambda x:1 if any(s in x for s in operatorClass) else 0)

#Drop extra columns
log_data_frame_cleaned = log_data_frame_cleaned.drop(['command'],axis=1)


log_data_frame_cleaned.head()

,t,type,ns,planSummary,planningTimeMicros,keysExamined,docsExamined,nBatches,cursorExhausted,numYields,...,selector_logical,selector_element,selector_evalutaion,selector_array,selector_bitwise,projection,misc,selector,standard_logical,all_operators
0,1.712243e+09,command,test_database.test_collection,EOF,83.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,1
1,1.712243e+09,command,test_database.test_collection,EOF,71.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,1
2,1.712243e+09,command,test_database.test_collection,EOF,42.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,1
3,1.712243e+09,command,test_database.test_collection,EOF,46.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,1
4,1.712243e+09,command,test_database.test_collection,EOF,83.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,1


In [63]:
log_data_frame_cleaned.dtypes

t                     float64
type                   object
ns                     object
planSummary            object
planningTimeMicros    float64
                       ...   
projection              int64
misc                    int64
selector                int64
standard_logical        int64
all_operators           int64
Length: 76, dtype: object

In [64]:
# Remove Columns with Constant Value
constant_columns = [col for col in log_data_frame_cleaned.columns if log_data_frame_cleaned[col].nunique() == 1]
log_data_frame_optimized = log_data_frame_cleaned.drop(columns=constant_columns)
log_data_frame_optimized.head()

,t,ns,planSummary,planningTimeMicros,cursorExhausted,queryFramework,reslen,locks,cpuNanos,durationMillis,...,selector_logical,selector_element,selector_evalutaion,selector_array,selector_bitwise,projection,misc,selector,standard_logical,all_operators
0,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,176371,0,...,1,1,1,1,1,1,1,1,1,1
1,1.712243e+09,test_database.test_collection,EOF,71.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,112781,0,...,1,1,1,1,1,1,1,1,1,1
2,1.712243e+09,test_database.test_collection,EOF,42.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,71093,0,...,1,1,1,1,1,1,1,1,1,1
3,1.712243e+09,test_database.test_collection,EOF,46.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,81173,0,...,1,1,1,1,1,1,1,1,1,1
4,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,141175,0,...,1,1,1,1,1,1,1,1,1,1


In [65]:
# Removed Columns

constant_columns

['type',
 'keysExamined',
 'docsExamined',
 'nBatches',
 'numYields',
 'nreturned',
 'storage',
 'remote',
 'protocol',
 '$db',
 '$and',
 '$nor',
 '$gte',
 '$lt',
 'lte',
 '$exists',
 '$expr',
 '$jsonSchema',
 '$text',
 '$all',
 '$bitsAllClear',
 '$bitsAllSet',
 '$bitsAnyClear',
 '$bitsAnySet',
 'meta',
 '$slice',
 '$rand',
 '$natural',
 '!=',
 '||',
 '!']

In [ ]:
# Save All

log_data_frame_cleaned.to_csv("../processed_datasets/log_data_cleaned.csv",encoding='utf-8')
log_data_frame_optimized.to_csv("../processed_datasets/log_data_cleaned_optimized.csv",encoding='utf-8')

OSError: Cannot save file into a non-existent directory: 'cleaned_data'